In [1]:
!python -V

Python 3.9.7


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [40]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [50]:
mlflow.set_experiment("homework 2")

2022/08/13 02:43:40 INFO mlflow.tracking.fluent: Experiment with name 'homework 2' does not exist. Creating a new experiment.


<Experiment: artifact_location='./mlruns/2', experiment_id='2', lifecycle_stage='active', name='homework 2', tags={}>

In [18]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [20]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')

df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')



In [21]:
len(df_train), len(df_val)

(73908, 61921)

In [22]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [23]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [24]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [25]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.47954063980395

In [27]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [45]:
with mlflow.start_run():

    mlflow.set_tag("developer", "Disha")
    
    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)
 
    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")
        

In [30]:
!pip install xgboost

In [34]:
import xgboost as xgb

In [33]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [35]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [36]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [46]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[17:48:58] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:11.85765                                                    
[1]	validation-rmse:8.39807                                                     
[2]	validation-rmse:7.23707                                                     
[3]	validation-rmse:6.85761                                                     
[4]	validation-rmse:6.71748                                                     
[5]	validation-rmse:6.66044                                                     
[6]	validation-rmse:6.63348                                                     
[7]	validation-rmse:6.61673                                                     
[8]	validation-rmse:6.61055                                                     
[9]	validation-rmse:6.60326                 

[98]	validation-rmse:6.51076                                                    
[99]	validation-rmse:6.51035                                                    
[100]	validation-rmse:6.51028                                                   
[101]	validation-rmse:6.51035                                                   
[102]	validation-rmse:6.51084                                                   
[103]	validation-rmse:6.51217                                                   
[104]	validation-rmse:6.51435                                                   
[105]	validation-rmse:6.51472                                                   
[106]	validation-rmse:6.51401                                                   
[107]	validation-rmse:6.51353                                                   
[108]	validation-rmse:6.51429                                                   
[109]	validation-rmse:6.51556                                                   
[110]	validation-rmse:6.5158

[73]	validation-rmse:6.39050                                                    
[74]	validation-rmse:6.38974                                                    
[75]	validation-rmse:6.38960                                                    
[76]	validation-rmse:6.38882                                                    
[77]	validation-rmse:6.38837                                                    
[78]	validation-rmse:6.38769                                                    
[79]	validation-rmse:6.38772                                                    
[80]	validation-rmse:6.38706                                                    
[81]	validation-rmse:6.38633                                                    
[82]	validation-rmse:6.38586                                                    
[83]	validation-rmse:6.38540                                                    
[84]	validation-rmse:6.38497                                                    
[85]	validation-rmse:6.38475

[174]	validation-rmse:6.37430                                                   
[175]	validation-rmse:6.37422                                                   
[176]	validation-rmse:6.37389                                                   
[177]	validation-rmse:6.37388                                                   
[178]	validation-rmse:6.37380                                                   
[179]	validation-rmse:6.37394                                                   
[180]	validation-rmse:6.37440                                                   
[181]	validation-rmse:6.37411                                                   
[182]	validation-rmse:6.37428                                                   
[183]	validation-rmse:6.37415                                                   
[184]	validation-rmse:6.37449                                                   
[185]	validation-rmse:6.37464                                                   
[186]	validation-rmse:6.3741

[44]	validation-rmse:6.52612                                                    
[45]	validation-rmse:6.52482                                                    
[46]	validation-rmse:6.52397                                                    
[47]	validation-rmse:6.52244                                                    
[48]	validation-rmse:6.52140                                                    
[49]	validation-rmse:6.52015                                                    
[50]	validation-rmse:6.51933                                                    
[51]	validation-rmse:6.51848                                                    
[52]	validation-rmse:6.51788                                                    
[53]	validation-rmse:6.51672                                                    
[54]	validation-rmse:6.51600                                                    
[55]	validation-rmse:6.51521                                                    
[56]	validation-rmse:6.51446

...

[462]	validation-rmse:6.39108                                                   
[463]	validation-rmse:6.39072                                                   
[464]	validation-rmse:6.39049                                                   
[465]	validation-rmse:6.39059                                                   
[466]	validation-rmse:6.39032                                                   
[467]	validation-rmse:6.39002                                                   
[468]	validation-rmse:6.39016                                                   
[469]	validation-rmse:6.39063                                                   
[470]	validation-rmse:6.39062                                                   
[471]	validation-rmse:6.39064                                                   
[472]	validation-rmse:6.38991                                                   
[473]	validation-rmse:6.38988                                                   
[474]	validation-rmse:6.3896

[563]	validation-rmse:6.38511                                                   
[564]	validation-rmse:6.38528                                                   
[565]	validation-rmse:6.38484                                                   
[566]	validation-rmse:6.38459                                                   
[567]	validation-rmse:6.38434                                                   
[568]	validation-rmse:6.38414                                                   
[569]	validation-rmse:6.38391                                                   
[570]	validation-rmse:6.38363                                                   
[571]	validation-rmse:6.38394                                                   
[572]	validation-rmse:6.38433                                                   
[573]	validation-rmse:6.38453                                                   
[574]	validation-rmse:6.38428                                                   
[575]	validation-rmse:6.3842

[664]	validation-rmse:6.38154                                                   
[665]	validation-rmse:6.38148                                                   
[666]	validation-rmse:6.38162                                                   
[667]	validation-rmse:6.38159                                                   
[668]	validation-rmse:6.38174                                                   
[669]	validation-rmse:6.38167                                                   
[670]	validation-rmse:6.38173                                                   
[671]	validation-rmse:6.38189                                                   
[672]	validation-rmse:6.38204                                                   
[673]	validation-rmse:6.38220                                                   
[674]	validation-rmse:6.38220                                                   
[675]	validation-rmse:6.38202                                                   
[676]	validation-rmse:6.3817

[84]	validation-rmse:6.44610                                                    
[85]	validation-rmse:6.44465                                                    
[86]	validation-rmse:6.44464                                                    
[87]	validation-rmse:6.44392                                                    
[88]	validation-rmse:6.44299                                                    
[89]	validation-rmse:6.44292                                                    
[90]	validation-rmse:6.44285                                                    
[91]	validation-rmse:6.44255                                                    
[92]	validation-rmse:6.44311                                                    
[93]	validation-rmse:6.44319                                                    
[94]	validation-rmse:6.44258                                                    
[95]	validation-rmse:6.44214                                                    
[96]	validation-rmse:6.44192

[185]	validation-rmse:6.43800                                                   
[186]	validation-rmse:6.43833                                                   
[187]	validation-rmse:6.43861                                                   
[188]	validation-rmse:6.43866                                                   
[189]	validation-rmse:6.43933                                                   
[190]	validation-rmse:6.43926                                                   
[191]	validation-rmse:6.43948                                                   
[192]	validation-rmse:6.43922                                                   
[193]	validation-rmse:6.43911                                                   
[194]	validation-rmse:6.43891                                                   
[195]	validation-rmse:6.43901                                                   
[196]	validation-rmse:6.43891                                                   
[197]	validation-rmse:6.4421

In [47]:
mlflow.xgboost.autolog(disable=True)

In [48]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[18:28:33] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:19.48427
[1]	validation-rmse:17.95632
[2]	validation-rmse:16.59113
[3]	validation-rmse:15.37412
[4]	validation-rmse:14.29011
[5]	validation-rmse:13.32801
[6]	validation-rmse:12.47570
[7]	validation-rmse:11.72140
[8]	validation-rmse:11.05888
[9]	validation-rmse:10.47582
[10]	validation-rmse:9.96482
[11]	validation-rmse:9.51784
[12]	validation-rmse:9.12621
[13]	validation-rmse:8.78521
[14]	validation-rmse:8.49037
[15]	validation-rmse:8.23332
[16]	validation-rmse:8.01179
[17]	validation-rmse:7.81822
[18]	validation-rmse:7.65208
[19]	validation-rmse:7.50742
[20]	validation-rmse:7.38234
[21]	validation-rmse:7.27382
[22]	validation-rmse:7.17865
[23]	validation-rmse:7.09606
[24]	validation-rmse:7.02481
[25]	validation-rmse:6.96336
[26]	validation-rms

[269]	validation-rmse:6.38814
[270]	validation-rmse:6.38786
[271]	validation-rmse:6.38755
[272]	validation-rmse:6.38727
[273]	validation-rmse:6.38673
[274]	validation-rmse:6.38660
[275]	validation-rmse:6.38649
[276]	validation-rmse:6.38615
[277]	validation-rmse:6.38595
[278]	validation-rmse:6.38584
[279]	validation-rmse:6.38558
[280]	validation-rmse:6.38529
[281]	validation-rmse:6.38509
[282]	validation-rmse:6.38479
[283]	validation-rmse:6.38455
[284]	validation-rmse:6.38428
[285]	validation-rmse:6.38410
[286]	validation-rmse:6.38377
[287]	validation-rmse:6.38341
[288]	validation-rmse:6.38321
[289]	validation-rmse:6.38288
[290]	validation-rmse:6.38241
[291]	validation-rmse:6.38218
[292]	validation-rmse:6.38186
[293]	validation-rmse:6.38155
[294]	validation-rmse:6.38099
[295]	validation-rmse:6.38072
[296]	validation-rmse:6.38058
[297]	validation-rmse:6.38034
[298]	validation-rmse:6.38019
[299]	validation-rmse:6.38001
[300]	validation-rmse:6.37985
[301]	validation-rmse:6.37969
[302]	vali

[543]	validation-rmse:6.33739
[544]	validation-rmse:6.33730
[545]	validation-rmse:6.33707
[546]	validation-rmse:6.33694
[547]	validation-rmse:6.33680
[548]	validation-rmse:6.33659
[549]	validation-rmse:6.33672
[550]	validation-rmse:6.33653
[551]	validation-rmse:6.33657
[552]	validation-rmse:6.33651
[553]	validation-rmse:6.33631
[554]	validation-rmse:6.33623
[555]	validation-rmse:6.33601
[556]	validation-rmse:6.33609
[557]	validation-rmse:6.33599
[558]	validation-rmse:6.33582
[559]	validation-rmse:6.33582
[560]	validation-rmse:6.33562
[561]	validation-rmse:6.33555
[562]	validation-rmse:6.33554
[563]	validation-rmse:6.33540
[564]	validation-rmse:6.33525
[565]	validation-rmse:6.33519
[566]	validation-rmse:6.33517
[567]	validation-rmse:6.33522
[568]	validation-rmse:6.33518
[569]	validation-rmse:6.33510
[570]	validation-rmse:6.33491
[571]	validation-rmse:6.33472
[572]	validation-rmse:6.33458
[573]	validation-rmse:6.33443
[574]	validation-rmse:6.33427
[575]	validation-rmse:6.33401
[576]	vali

[817]	validation-rmse:6.31368
[818]	validation-rmse:6.31365
[819]	validation-rmse:6.31363
[820]	validation-rmse:6.31342
[821]	validation-rmse:6.31328
[822]	validation-rmse:6.31317
[823]	validation-rmse:6.31322
[824]	validation-rmse:6.31316
[825]	validation-rmse:6.31318
[826]	validation-rmse:6.31317
[827]	validation-rmse:6.31316
[828]	validation-rmse:6.31309
[829]	validation-rmse:6.31290
[830]	validation-rmse:6.31278
[831]	validation-rmse:6.31271
[832]	validation-rmse:6.31266
[833]	validation-rmse:6.31270
[834]	validation-rmse:6.31273
[835]	validation-rmse:6.31271
[836]	validation-rmse:6.31268
[837]	validation-rmse:6.31259
[838]	validation-rmse:6.31253
[839]	validation-rmse:6.31239
[840]	validation-rmse:6.31230
[841]	validation-rmse:6.31226
[842]	validation-rmse:6.31215
[843]	validation-rmse:6.31216
[844]	validation-rmse:6.31220
[845]	validation-rmse:6.31214
[846]	validation-rmse:6.31200
[847]	validation-rmse:6.31178
[848]	validation-rmse:6.31168
[849]	validation-rmse:6.31149
[850]	vali

/Users/dishamendiratta/opt/anaconda3/envs/exp-tracking-env/lib/python3.10/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [49]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

2022/08/12 19:25:23 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: could not write to './mlruns/1/4be1d62447b542d782d5bed6e22652e5/artifacts/model/model.pkl': No space left on device
/Users/dishamendiratta/opt/anaconda3/envs/exp-tracking-env/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
